In [1]:
import requests
import json
from pandas import DataFrame
from matplotlib import pyplot as plt
import datetime as dt
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import time
import urllib
import schedule

# 3,4번 코드

In [2]:
def update_covid19_data():
    HOSTNAME = "localhost"
    PORT = 3306
    USERNAME = "root"
    PASSWORD = "root"
    DATABASE = "covid_19"
    CHARSET1 = "utf8"
    CHARSET2 = "utf-8"

    json_url = 'http://www.itpaper.co.kr/demo/covid19/now.php'

    user_agent ="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
    header_info ={'User-agent': user_agent,'referer':None}

    r = requests.get(json_url, headers = header_info)

    if r.status_code != 200:
        err_msg = '%d %s 에러가 발생했습니다.' %(r.status_code, r.reason)
        raise Exception(err_msg)

    r.encoding = 'utf-8'
    json_dict = json.loads(r.text)
    df = DataFrame(json_dict['state'])

    now = dt.datetime.now()
    hour = now.hour

    if hour < 23:
        now += dt.timedelta(days = -1)

    time_str = now.strftime('%Y-%m-%d')
    df['reg_date'] = time_str

    con_str_fmt	="mysql+mysqldb://{0}:{1}@{2}:{3}/{4}?charset={5}"
    con_str = con_str_fmt.format(USERNAME, urllib.parse.quote(PASSWORD), HOSTNAME, PORT, DATABASE,
    CHARSET1)
    pymysql.install_as_MySQLdb()
    import MySQLdb

    engine = create_engine(con_str,encoding=CHARSET2)
    conn = engine.connect()

    engine.execute(text('DELETE FROM covid19_state WHERE reg_date = :date'),{'date':time_str})

    df.to_sql(name='covid19_state', con=conn, if_exists='append',index=False)
    print('excute_success')
    
schedule.every(10).seconds.do(update_covid19_data)

schedule.every().days.at('10:00').do(update_covid19_data)

Every 1 day at 10:00:00 do update_covid19_data() (last run: [never], next run: 2021-08-06 10:00:00)

In [9]:
test = 0 # 테스트용 변수
 
while test < 50:
    schedule.run_pending()
    time.sleep(1)
    test += 1

excute_success
excute_success
excute_success
excute_success
excute_success
